# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Dependencies](#dependencies)**
2. **[Tables](#tables)**
2. **[Views](#views)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/common/schemas.ipynb",
        "src/common/conversions.ipynb",
    ]
}
```

# **TABLES**

---


### DESTINATIONS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY]') AND TYPE = 'SO')
CREATE SEQUENCE SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE [NAME] = 'RIVM_VACCINATIONS_PARTLY_FULLY')
CREATE TABLE [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY](
    [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY]),
    WEEK_START DATETIME NULL,
    WEEK_END DATETIME NULL,
    PARTIALLY_VACCINATED INT NULL,
    FULLY_VACCINATED INT NULL,
    PARTIALLY_OR_FULLY_VACCINATED INT NULL,
    [BOOSTER1] INT NULL, 
    [BOOSTER1_INCREMENT] INT NULL,
    [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
GO

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_RIVM_VACCINATIONS_PARTLY_FULLY_DLI')
CREATE NONCLUSTERED INDEX IX_DEST_RIVM_VACCINATIONS_PARTLY_FULLY_DLI
ON VWSDEST.RIVM_VACCINATIONS_PARTLY_FULLY (DATE_LAST_INSERTED)
INCLUDE (WEEK_START);
GO

# **VIEWS**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER VIEW VWSDEST.V_VACCINATIONS_PARTLY_FULLY
AS
    WITH BASE_CTE AS (
        SELECT
            PARTIALLY_VACCINATED,
            FULLY_VACCINATED,
            PARTIALLY_OR_FULLY_VACCINATED,
            BOOSTER1,
            DBO.CONVERT_DATETIME_TO_UNIX(WEEK_START) AS DATE_START_UNIX,
            DBO.CONVERT_DATETIME_TO_UNIX(WEEK_END) AS DATE_END_UNIX,
            DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
      FROM [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY]
      WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY])
        AND WEEK_START >= '2021-01-04 00:00:00.000' -- VACCINATION STARTED IN THE WEEK OF 4-1-2021
    )

    -- FILTER OUT:
    -- 1. ANY INCOMPLETE WEEKS / FUTURE WEEKS
    SELECT
        PARTIALLY_VACCINATED,
        FULLY_VACCINATED,
        PARTIALLY_OR_FULLY_VACCINATED,
        BOOSTER1,
        DATE_START_UNIX,
        DATE_END_UNIX,
        DATE_OF_INSERTION_UNIX
    FROM BASE_CTE
    WHERE DATE_OF_INSERTION_UNIX > DATE_END_UNIX
GO